## Import Packages

In [ ]:
#https://docs.bokeh.org/en/latest/docs/gallery.html
#https://docs.bokeh.org/en/latest/docs/user_guide/plotting.html
#https://docs.bokeh.org/en/latest/docs/gallery/slider.html
#https://github.com/bokeh/bokeh/tree/branch-2.3/examples/app/weather
#https://jeffreyalevy.com/


In [2]:
import requests
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import urllib
import time

from scipy.interpolate import InterpolatedUnivariateSpline
from scipy.interpolate import make_interp_spline, BSpline
from scipy import interpolate

from bokeh.plotting import figure, show, output_file, save
from bokeh.layouts import gridplot
from bokeh.models import HoverTool, SingleIntervalTicker, LinearAxis
from ipywidgets import interact
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

## Define Paths

In [3]:
start = time.time()
end = time.time()

path = '/Users/marksheppard/Documents/GitHub/Python Project/'
psz_url = 'http://gabriel-zucman.eu/files/PSZ2018MainData.xlsx'
as_url = 'http://davidsplinter.com/AutenSplinter-IncomeIneq.xlsx'
urllib.request.urlretrieve(psz_url, "piketty-saez-zucman.xlsx")
urllib.request.urlretrieve(as_url, "auten-splinter.xlsx")

print("Downloading the data required", (end - start), "seconds.")

In [4]:
start_ = time.time()
end_ = time.time()
psz_df = pd.read_excel(psz_url, 'Data', usecols=('A:AN, AQ:BJ, BM:CY, DB:EO, EQ:FY, GA:GN, GQ:HD, HG:HR, HT:IM, IO:IP, IS:JE, JG:JO'), header=[2]).drop([0,1]).rename(columns={"Series": "Year"})
psz_df.drop(psz_df.tail(11).index,inplace=True)
as_df = pd.read_excel(as_url, 'F-A1', header=[32], usecols=('A:E, G:J')).rename(columns={'Unnamed: 0': "Year"}).drop([56,57])                                             
print("Cleaning the data required", (end_ - start_), "seconds.")                                                                      

Cleaning the data required 3.790855407714844e-05 seconds.


In [6]:
df = psz_df.merge(as_df, how='outer')
print("Merge complete")  

Merge complete


In [96]:
# determine best fit line
par = np.polyfit(xnew[130:-2], psz_smooth[130:-2]*100, 1, full=True)
slope=par[0][0]
intercept=par[0][1]
y_predicted = [slope*i + intercept  for i in xnew[130:-2]]


# determine best fit line
par_as = np.polyfit(xnew[80:-2], as_smooth[80:-2]*100, 1, full=True)
slope_as=par_as[0][0]
intercept_as=par_as[0][1]
y_predicted_as = [slope_as*i + intercept_as  for i in xnew]



hover = HoverTool(
        tooltips="""
        <div style="background-color: rgba(0, 0, 0, 0.0);>
            <div style="background-color: rgba(2, 0, 0, 0.0);>
                <span style="font-size: 8px;">Year</span>
                <span style="font-size: 8px; color: #7c7c7c;">$x{2f}</span>
                </br>
                <span style="font-size: 8px;">Value</span>
                <span style="font-size: 8px; color: #7c7c7c;">$y{2f}%</span>
            </div>
        </div>
        """
    )
plot = figure(x_axis_label='Year', 
              y_axis_label='Share of National Income',
              tools=[hover],
              plot_width=700, 
              plot_height=500,
              title="Top 10% Share of National Income")

plot.outline_line_color = None
plot.xaxis.major_tick_line_color = None  
plot.yaxis.major_tick_line_color = None
plot.xaxis.minor_tick_line_color = None  
plot.yaxis.minor_tick_line_color = None
plot.xgrid.grid_line_color = None
plot.ygrid.grid_line_dash = [2, 2]
plot.yaxis.axis_line_color = "White"
plot.xaxis.axis_line_color = "#7c7c7c"
plot.toolbar.logo = None



year = list(df['Year'])
psz_top10 = list(df['Memo: Top 10% fiscal income per tax unit, incl. KG'].fillna(df['Memo: Top 10% fiscal income per tax unit, incl. KG']
                                                                                 .mean()))

as_top10 = list(df['Pre-tax after-transfer income'].fillna(df['Pre-tax after-transfer income']
                                                                                 .mean()))

xnew = np.linspace(min(year), max(year), 300)
psz_spl = make_interp_spline(year, psz_top10, k=3)
as_spl = make_interp_spline(year, as_top10, k=3)

psz_smooth = psz_spl(xnew)
as_smooth = as_spl(xnew)

plot.line(xnew[:-2], psz_smooth[:-2]*100, line_width=1.5, color='#9ebed2', legend_label="Piketty, Saez, & Zucman (PSZ)",
         muted_alpha=0.2)
plot.line(xnew[130:-2], as_smooth[130:-2]*100, line_width=1.5, color="#5787ae", legend_label="Auten & Splinter (AS)",
         muted_alpha=0.2)




plot.line(xnew[130:], y_predicted, line_width=1, color="#9ebed2", line_dash="dotted", muted_alpha=0.2,
          legend_label='PSZ Slope Coefficient: '+str(round(slope,2)))


plot.line(xnew[130:], y_predicted_as,line_width=1, color="#5787ae", line_dash="dotted", muted_alpha=0.2,
          legend_label='AS Slope Coefficient: '+str(round(slope_as,2)))





plot.xaxis.ticker = SingleIntervalTicker(interval=10)
plot.yaxis.ticker = SingleIntervalTicker(interval=2)

plot.legend.border_line_width = 0
plot.legend.title = 'Researchers'
plot.legend.title_text_font_style = "bold"
plot.legend.title_text_font_size = "15px"
plot.legend.label_text_font_size = "10px" 
plot.legend.location = "bottom_left"
plot.legend.click_policy="mute"
plot.legend.background_fill_alpha = 0.0

show(plot)

output_file("Sample.html")
save(plot)

'/Users/marksheppard/Documents/GitHub/PikettySaezZucmanAutenSplinter/Sample.html'

In [11]:
# determine best fit line
par = np.polyfit(xnew, psz_smooth, 1, full=True)
slope=par[0][0]
intercept=par[0][1]
y_predicted = [slope*i + intercept  for i in xnew]

fig=figure()
fig.circle(xnew,psz_smooth)
fig.line(xnew,y_predicted,color='grey',legend='psz_smooth='+ str(round(slope,2))+'xnew+'+str(round(intercept,2)))
show(fig)